In [20]:
# on server: 'screen' ,then start script
# use 'strg+a d' to return to terminal
# use 'screen -r' to return to screen
import numpy as np
import json
import os

from keras.layers.embeddings import Embedding
from keras.layers import Concatenate
from keras.models import Model
from keras.layers import Input,Dense,LSTM
from keras.utils import plot_model
# os.environ['CUDA_VISIBLE_DEVICES']='0'

import process_data as pd

In [21]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [22]:
# https://towardsdatascience.com/nlp-sequence-to-sequence-networks-part-1-processing-text-data-d141a5643b72
path='testpath/'
# rnn parameters
hidden_size = 100 #100 is the standard
batch_size = 512 #for the training on the GPU this to be has to very large, otherwise the GPU is used very inefficiently
epochs = 100

size=10000

#glove embedding parameters
glove_dir = '../glove/glove.6B.100d.txt'
embedding_dim = 100

In [23]:
#open SQuAD-dataset and extract the relevant data from the json-file
#to a easier readable/accessible dictionary
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=paragraph['context']
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            #add a bos and eos token to the target
            if (qas['is_impossible']==False):
                a=context.lower()
                b=qas['question'].lower()
                c=qas['answers'][0]['text'].lower()
                
                train_new['context'].append(a)
                train_new['question'].append(b)
                train_new['answer'].append('START_ '+c+' _END')
print(len(train_new['context']))
print(len(train_new['question']))
print(len(train_new['answer']))

86821
86821
86821


In [25]:
context=train_new['context'][size:size*2]
question=train_new['question'][size:size*2]
answer=train_new['answer'][size:size*2]
data=[context,question,answer]
input_data=pd.process_data(data)

length of vocab:  34269 14031 9109
max length of data: 426 37 29
length of data:  10000 10000 10000
shape of the input for the model:  (10000, 426) (10000, 37) (10000, 29) (10000, 29, 9109)
dict_keys(['len_vocab', 'token_to_int', 'encoder_input', 'decoder_input', 'int_to_token'])
encoder_input keys:  dict_keys(['question_encoder_input', 'context_encoder_input'])
decoder_input keys:  dict_keys(['answer_decoder_target', 'answer_decoder_input'])
len_vocab:  dict_keys(['context_len_vocab', 'question_len_vocab', 'answer_len_vocab'])
token_to_int:  dict_keys(['question_token_to_int', 'answer_token_to_int', 'context_token_to_int'])


In [26]:
context_encoder_input=input_data['encoder_input']['context_encoder_input']
question_encoder_input=input_data['encoder_input']['question_encoder_input']
answer_decoder_input=input_data['decoder_input']['answer_decoder_input']
answer_decoder_target=input_data['decoder_input']['answer_decoder_target']

context_len_vocab=input_data['len_vocab']['context_len_vocab']
question_len_vocab=input_data['len_vocab']['question_len_vocab']
answer_len_vocab=input_data['len_vocab']['answer_len_vocab']

context_token_to_int=input_data['token_to_int']['context_token_to_int']
question_token_to_int=input_data['token_to_int']['question_token_to_int']
answer_token_to_int=input_data['token_to_int']['answer_token_to_int']

answer_int_to_token=input_data['int_to_token']['answer_int_to_token']

In [27]:
#FIX_ME: add glove download
# https://nlp.stanford.edu/projects/glove/
#get glove embeddings
embeddings_index = {}
f = open(glove_dir)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [28]:
#extract the glove-embedding to a matrix
context_embedding_matrix = np.zeros((context_len_vocab, embedding_dim))
for word, i in context_token_to_int.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        context_embedding_matrix[i] = embedding_vector

question_embedding_matrix = np.zeros((question_len_vocab, embedding_dim))
for word, i in question_token_to_int.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        question_embedding_matrix[i] = embedding_vector

answer_embedding_matrix = np.zeros((answer_len_vocab, embedding_dim))
for word, i in answer_token_to_int.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        answer_embedding_matrix[i] = embedding_vector
print(np.shape(context_embedding_matrix),np.shape(question_embedding_matrix),np.shape(answer_embedding_matrix))

(34269, 100) (14031, 100) (9109, 100)


In [29]:
# https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7
# encoder
context_encoder_inputs = Input(shape=(None,))
context_embedding_layer = Embedding(context_len_vocab, 
                        embedding_dim,weights=[context_embedding_matrix],trainable=False)
context_embedding=context_embedding_layer(context_encoder_inputs)

context_decoder_lstm = LSTM(embedding_dim,return_state=True)
context_x, context_state_h, context_state_c = context_decoder_lstm(context_embedding)
context_encoder_states = [context_state_h, context_state_c]


question_encoder_inputs = Input(shape=(None,))
question_embedding_layer = Embedding(question_len_vocab, 
                    embedding_dim,weights=[question_embedding_matrix],trainable=False)
question_embedding=question_embedding_layer(question_encoder_inputs)

question_decoder_lstm = LSTM(embedding_dim,return_state=True)
question_x, question_state_h, question_state_c = question_decoder_lstm(question_embedding)
question_encoder_states = [question_state_h, question_state_c]


encoder_state_h=Concatenate()([context_state_h,question_state_h])
encoder_state_c=Concatenate()([context_state_c,question_state_c])
concat_encoder_states=[encoder_state_h,encoder_state_c]

# decoder #################################
decoder_inputs = Input(shape=(None,))
answer_embedding_layer = Embedding(answer_len_vocab, 
                             embedding_dim,weights=[answer_embedding_matrix])
answer_embedding = answer_embedding_layer(decoder_inputs)

decoder_lstm = LSTM(embedding_dim*2, return_sequences=True,return_state=True)
decoder_lstm_output,_,_ = decoder_lstm(answer_embedding, initial_state=concat_encoder_states)

decoder_dense = Dense(answer_len_vocab, activation='softmax')
decoder_output = decoder_dense(decoder_lstm_output)

model = Model([context_encoder_inputs,question_encoder_inputs, decoder_inputs], decoder_output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    3426900     input_6[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    1403100     input_7[0][0]                    
__________________________________________________________________________________________________
input_8 (I

In [10]:
# model.fit([context_encoder_input,
#            question_encoder_input, 
#            answer_decoder_input], 
#           answer_decoder_target,
#           batch_size=batch_size,
#           epochs=epochs,)

In [11]:
model.load_weights('models/baseline_model.h5')

In [12]:
# define the encoder model 
encoder_model = Model([context_encoder_inputs,question_encoder_inputs],concat_encoder_states)
encoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    3329500     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    1282800     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [13]:
decoder_state_input_h = Input(shape=(hidden_size*2,))
decoder_state_input_c = Input(shape=(hidden_size*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= answer_embedding_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2,)

decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    854100      input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
lstm_3 (LS

In [14]:
# encoder_model.predict([input_data['encoder_input']['encoder_input_context'][0:1],
#                        input_data['encoder_input']['encoder_input_question'][0:1]])

In [15]:
def decode_sequence(context_input_seq,question_input_seq,):
    # Encode the input as state vectors.
    states_value = encoder_model.predict([context_input_seq,question_input_seq])
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = answer_token_to_int['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = answer_int_to_token[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
    return decoded_sentence[:-4] #[:-4] removes the end-of-sequence token

In [17]:
for seq_index in range(100):
    context_input_seq = input_data['encoder_input']['context_encoder_input'][seq_index:seq_index+1]
    question_input_seq = input_data['encoder_input']['question_encoder_input'][seq_index:seq_index+1]
    
    decoded_sentence = decode_sequence(context_input_seq,question_input_seq)
    print('-')
    print('context: ',train_new['context'][seq_index:seq_index+1],'\n')
    print('question: ',train_new['question'][seq_index:seq_index+1],'\n')
    print('answer: ', decoded_sentence)

-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best-selling girl groups of all time. their hiatus saw the release of beyoncé\'s debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles "crazy in love" and "baby boy".'] 

question:  ['when did beyonce start becoming popular?'] 

answer:   january 7, 2012 
-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she

-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best-selling girl groups of all time. their hiatus saw the release of beyoncé\'s debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles "crazy in love" and "baby boy".'] 

question:  ['how many grammy awards did beyoncé win for her first solo album?'] 

answer:   five 
-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in housto

-
context:  ['following the disbandment of destiny\'s child in june 2005, she released her second solo album, b\'day (2006), which contained hits "déjà vu", "irreplaceable", and "beautiful liar". beyoncé also ventured into acting, with a golden globe-nominated performance in dreamgirls (2006), and starring roles in the pink panther (2006) and obsessed (2009). her marriage to rapper jay z and portrayal of etta james in cadillac records (2008) influenced her third album, i am... sasha fierce (2008), which saw the birth of her alter-ego sasha fierce and earned a record-setting six grammy awards in 2010, including song of the year for "single ladies (put a ring on it)". beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. her critically acclaimed fifth studio album, beyoncé (2013), was distinguished from previous releases by its experimental production a

-
context:  ['a self-described "modern-day feminist", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny\'s child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award\'s history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of the decade, the top female artist of the 2000s and their artist of the millennium in 2011. time listed her among the 100 most influential people in the world in 2013 and 2014.

-
context:  ['a self-described "modern-day feminist", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny\'s child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award\'s history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of the decade, the top female artist of the 2000s and their artist of the millennium in 2011. time listed her among the 100 most influential people in the world in 2013 and 2014.

...

-
context:  ["at age eight, beyoncé and childhood friend kelly rowland met latavia roberson while in an audition for an all-girl entertainment group. they were placed into a group with three other girls as girl's tyme, and rapped and danced on the talent show circuit in houston. after seeing the group, r&b producer arne frager brought them to his northern california studio and placed them in star search, the largest talent show on national tv at the time. girl's tyme failed to win, and beyoncé later said the song they performed was not good. in 1995 beyoncé's father resigned from his job to manage the group. the move reduced beyoncé's family's income by half, and her parents were forced to move into separated apartments. mathew cut the original line-up to four and the group continued performing as an opening act for other established r&b girl groups. the girls auditioned before record labels and were finally signed to elektra records, moving to atlanta records briefly to work on their 

-
context:  ["at age eight, beyoncé and childhood friend kelly rowland met latavia roberson while in an audition for an all-girl entertainment group. they were placed into a group with three other girls as girl's tyme, and rapped and danced on the talent show circuit in houston. after seeing the group, r&b producer arne frager brought them to his northern california studio and placed them in star search, the largest talent show on national tv at the time. girl's tyme failed to win, and beyoncé later said the song they performed was not good. in 1995 beyoncé's father resigned from his job to manage the group. the move reduced beyoncé's family's income by half, and her parents were forced to move into separated apartments. mathew cut the original line-up to four and the group continued performing as an opening act for other established r&b girl groups. the girls auditioned before record labels and were finally signed to elektra records, moving to atlanta records briefly to work on their 

-
context:  ['the group changed their name to destiny\'s child in 1996, based upon a passage in the book of isaiah. in 1997, destiny\'s child released their major label debut song "killing time" on the soundtrack to the 1997 film, men in black. the following year, the group released their self-titled debut album, scoring their first major hit "no, no, no". the album established the group as a viable act in the music industry, with moderate sales and winning the group three soul train lady of soul awards for best r&b/soul album of the year, best r&b/soul or rap new artist, and best r&b/soul single for "no, no, no". the group released their multi-platinum second album the writing\'s on the wall in 1999. the record features some of the group\'s most widely known songs such as "bills, bills, bills", the group\'s first number-one single, "jumpin\' jumpin\'" and "say my name", which became their most successful song at the time, and would remain one of their signature songs. "say my name" wo

-
context:  ['the group changed their name to destiny\'s child in 1996, based upon a passage in the book of isaiah. in 1997, destiny\'s child released their major label debut song "killing time" on the soundtrack to the 1997 film, men in black. the following year, the group released their self-titled debut album, scoring their first major hit "no, no, no". the album established the group as a viable act in the music industry, with moderate sales and winning the group three soul train lady of soul awards for best r&b/soul album of the year, best r&b/soul or rap new artist, and best r&b/soul single for "no, no, no". the group released their multi-platinum second album the writing\'s on the wall in 1999. the record features some of the group\'s most widely known songs such as "bills, bills, bills", the group\'s first number-one single, "jumpin\' jumpin\'" and "say my name", which became their most successful song at the time, and would remain one of their signature songs. "say my name" wo

-
context:  ["letoya luckett and roberson became unhappy with mathew's managing of the band and eventually were replaced by farrah franklin and michelle williams. beyoncé experienced depression following the split with luckett and roberson after being publicly blamed by the media, critics, and blogs for its cause. her long-standing boyfriend left her at this time. the depression was so severe it lasted for a couple of years, during which she occasionally kept herself in her bedroom for days and refused to eat anything. beyoncé stated that she struggled to speak about her depression because destiny's child had just won their first grammy award and she feared no one would take her seriously. beyoncé would later speak of her mother as the person who helped her fight it. franklin was dismissed, leaving just beyoncé, rowland, and williams."] 

question:  ["who replaced luckett and roberson in destiny's child?"] 

answer:   farrah franklin and michelle williams. 
-
context:  ["letoya luckett

In [18]:
context=train_new['context'][size:size*2]
question=train_new['question'][size:size*2]
answer=train_new['answer'][size:size*2]
data=[context,question,answer]
input_data=pd.process_data(data)

context_encoder_input=input_data['encoder_input']['context_encoder_input']
question_encoder_input=input_data['encoder_input']['question_encoder_input']
answer_decoder_input=input_data['decoder_input']['answer_decoder_input']
answer_decoder_target=input_data['decoder_input']['answer_decoder_target']

context_len_vocab=input_data['len_vocab']['context_len_vocab']
question_len_vocab=input_data['len_vocab']['question_len_vocab']
answer_len_vocab=input_data['len_vocab']['answer_len_vocab']

context_token_to_int=input_data['token_to_int']['context_token_to_int']
question_token_to_int=input_data['token_to_int']['question_token_to_int']
answer_token_to_int=input_data['token_to_int']['answer_token_to_int']

answer_int_to_token=input_data['int_to_token']['answer_int_to_token']

length of vocab:  34269 14031 9109
max length of data: 426 37 29
length of data:  10000 10000 10000
shape of the input for the model:  (10000, 426) (10000, 37) (10000, 29) (10000, 29, 9109)
dict_keys(['len_vocab', 'token_to_int', 'encoder_input', 'decoder_input', 'int_to_token'])
encoder_input keys:  dict_keys(['question_encoder_input', 'context_encoder_input'])
decoder_input keys:  dict_keys(['answer_decoder_target', 'answer_decoder_input'])
len_vocab:  dict_keys(['context_len_vocab', 'question_len_vocab', 'answer_len_vocab'])
token_to_int:  dict_keys(['question_token_to_int', 'answer_token_to_int', 'context_token_to_int'])


In [19]:
for seq_index in range(100):
    context_input_seq = input_data['encoder_input']['context_encoder_input'][seq_index:seq_index+1]
    question_input_seq = input_data['encoder_input']['question_encoder_input'][seq_index:seq_index+1]
    
    decoded_sentence = decode_sequence(context_input_seq,question_input_seq)
    print('-')
    print('context: ',train_new['context'][seq_index:seq_index+1],'\n')
    print('question: ',train_new['question'][seq_index:seq_index+1],'\n')
    print('answer: ', decoded_sentence)

-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best-selling girl groups of all time. their hiatus saw the release of beyoncé\'s debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles "crazy in love" and "baby boy".'] 

question:  ['when did beyonce start becoming popular?'] 

answer:   smooth 1923 50.6% 50.6% 50.6% acoustic nationalism m
-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. bo

-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best-selling girl groups of all time. their hiatus saw the release of beyoncé\'s debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles "crazy in love" and "baby boy".'] 

question:  ['when did beyoncé release dangerously in love?'] 

answer:   flysch 50.6% 50.6% 50.6% 50.6% acoustic event 50.6% 5
-
context:  ['beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actre

-
context:  ['following the disbandment of destiny\'s child in june 2005, she released her second solo album, b\'day (2006), which contained hits "déjà vu", "irreplaceable", and "beautiful liar". beyoncé also ventured into acting, with a golden globe-nominated performance in dreamgirls (2006), and starring roles in the pink panther (2006) and obsessed (2009). her marriage to rapper jay z and portrayal of etta james in cadillac records (2008) influenced her third album, i am... sasha fierce (2008), which saw the birth of her alter-ego sasha fierce and earned a record-setting six grammy awards in 2010, including song of the year for "single ladies (put a ring on it)". beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. her critically acclaimed fifth studio album, beyoncé (2013), was distinguished from previous releases by its experimental production a

-
context:  ['a self-described "modern-day feminist", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny\'s child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award\'s history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of the decade, the top female artist of the 2000s and their artist of the millennium in 2011. time listed her among the 100 most influential people in the world in 2013 and 2014.

-
context:  ['a self-described "modern-day feminist", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny\'s child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award\'s history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of the decade, the top female artist of the 2000s and their artist of the millennium in 2011. time listed her among the 100 most influential people in the world in 2013 and 2014.

...

-
context:  ["at age eight, beyoncé and childhood friend kelly rowland met latavia roberson while in an audition for an all-girl entertainment group. they were placed into a group with three other girls as girl's tyme, and rapped and danced on the talent show circuit in houston. after seeing the group, r&b producer arne frager brought them to his northern california studio and placed them in star search, the largest talent show on national tv at the time. girl's tyme failed to win, and beyoncé later said the song they performed was not good. in 1995 beyoncé's father resigned from his job to manage the group. the move reduced beyoncé's family's income by half, and her parents were forced to move into separated apartments. mathew cut the original line-up to four and the group continued performing as an opening act for other established r&b girl groups. the girls auditioned before record labels and were finally signed to elektra records, moving to atlanta records briefly to work on their 

-
context:  ["at age eight, beyoncé and childhood friend kelly rowland met latavia roberson while in an audition for an all-girl entertainment group. they were placed into a group with three other girls as girl's tyme, and rapped and danced on the talent show circuit in houston. after seeing the group, r&b producer arne frager brought them to his northern california studio and placed them in star search, the largest talent show on national tv at the time. girl's tyme failed to win, and beyoncé later said the song they performed was not good. in 1995 beyoncé's father resigned from his job to manage the group. the move reduced beyoncé's family's income by half, and her parents were forced to move into separated apartments. mathew cut the original line-up to four and the group continued performing as an opening act for other established r&b girl groups. the girls auditioned before record labels and were finally signed to elektra records, moving to atlanta records briefly to work on their 

-
context:  ['the group changed their name to destiny\'s child in 1996, based upon a passage in the book of isaiah. in 1997, destiny\'s child released their major label debut song "killing time" on the soundtrack to the 1997 film, men in black. the following year, the group released their self-titled debut album, scoring their first major hit "no, no, no". the album established the group as a viable act in the music industry, with moderate sales and winning the group three soul train lady of soul awards for best r&b/soul album of the year, best r&b/soul or rap new artist, and best r&b/soul single for "no, no, no". the group released their multi-platinum second album the writing\'s on the wall in 1999. the record features some of the group\'s most widely known songs such as "bills, bills, bills", the group\'s first number-one single, "jumpin\' jumpin\'" and "say my name", which became their most successful song at the time, and would remain one of their signature songs. "say my name" wo

-
context:  ['the group changed their name to destiny\'s child in 1996, based upon a passage in the book of isaiah. in 1997, destiny\'s child released their major label debut song "killing time" on the soundtrack to the 1997 film, men in black. the following year, the group released their self-titled debut album, scoring their first major hit "no, no, no". the album established the group as a viable act in the music industry, with moderate sales and winning the group three soul train lady of soul awards for best r&b/soul album of the year, best r&b/soul or rap new artist, and best r&b/soul single for "no, no, no". the group released their multi-platinum second album the writing\'s on the wall in 1999. the record features some of the group\'s most widely known songs such as "bills, bills, bills", the group\'s first number-one single, "jumpin\' jumpin\'" and "say my name", which became their most successful song at the time, and would remain one of their signature songs. "say my name" wo

-
context:  ["letoya luckett and roberson became unhappy with mathew's managing of the band and eventually were replaced by farrah franklin and michelle williams. beyoncé experienced depression following the split with luckett and roberson after being publicly blamed by the media, critics, and blogs for its cause. her long-standing boyfriend left her at this time. the depression was so severe it lasted for a couple of years, during which she occasionally kept herself in her bedroom for days and refused to eat anything. beyoncé stated that she struggled to speak about her depression because destiny's child had just won their first grammy award and she feared no one would take her seriously. beyoncé would later speak of her mother as the person who helped her fight it. franklin was dismissed, leaving just beyoncé, rowland, and williams."] 

question:  ['who supported beyonce through her depression?'] 

answer:   smooth affect 50.6% smooth peremptory rosalia commu
-
context:  ["letoya luc

In [ ]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# plot_model(model, to_file=path+'/model.png')

In [ ]:
# print('save model')
# if not os.path.isdir(path):
#     os.makedirs(path)
# model.save_weights(path+str('baseline_model.h5')) #save weights
# model_json = model.to_json()
# with open(path+str('baseline_model.json'),'w') as json_file:
#     json_file.write(model_json)